### 挂载 google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### 转到盘符

In [3]:
%cd /content/drive/My\ Drive/project/seq2seq_chatbot

/content/drive/My Drive/project/seq2seq_chatbot


### 训练

In [0]:
import random
import pickle
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sequence_to_sequence import SequenceToSequence
from data_utils import batch_flow_bucket as batch_flow
from word_sequence import WordSequence
from threadedgenerator import ThreadedGenerator
import os
import json

def train(params):
  x_data, y_data = pickle.load(open('./data/chatbot.pkl', 'rb'))
  ws = pickle.load(open('./data/ws.pkl', 'rb'))
    
  n_epoch = 2
  batch_size=128
  steps = int(len(x_data) / batch_size) +1
  config = tf.ConfigProto(
    allow_soft_placement=True,
    log_device_placement=False
  )

  save_path = 'model/s2s_chatbot_anti.ckpt'

  tf.reset_default_graph()
  with tf.Graph().as_default():
    random.seed(0)
    np.random.seed(0)
    tf.set_random_seed(0)
 
    with tf.Session(config=config) as sess:
      global_step = tf.Variable(0, name='global_step', trainable=False)

      model = SequenceToSequence(
        input_vocab_size=len(ws),
        target_vocab_size = len(ws),
        batch_size = batch_size,
        **params
      )
    
      init = tf.global_variables_initializer()   

      sess.run(init)  
             
      flow = ThreadedGenerator(
        batch_flow([x_data, y_data], ws, batch_size, add_end=[False, True]),
        queue_maxsize=30
      )

      dummy_encoder_inputs = np.array([
        np.array([WordSequence.PAD]) for _ in range(batch_size)
      ])

      dummy_encoder_inputs_length = np.array([1] * batch_size)
      
      ckpt = tf.train.latest_checkpoint('./model')  
      if not ckpt is None:
        model.load(sess, ckpt)
        
      start = global_step.eval()  
      for epoch in range(1, n_epoch+1):
        costs = []
        bar = tqdm(range(start, steps),
          total = steps,
          desc = 'epoch {}, loss=0.000000'.format(epoch),
          initial = start
        )

        for i in bar:
          print("当前：" + str(i))
          x, xl, y, yl = next(flow)
          x = np.flip(x, axis=1)

          add_loss = model.train(
            sess,
            dummy_encoder_inputs,
            dummy_encoder_inputs_length,
            y, yl, loss_only=True
          )

          add_loss *= -0.5

          cost, lr = model.train(sess, x, xl, y, yl,
            return_lr=True,
            add_loss=add_loss
          )
          costs.append(cost)         
          bar.set_description('epoch {} loss={:.6f} lr={:.6f}'.format(epoch, np.mean(costs), lr))
          global_step.assign(i).eval() # set and update(eval) global_step with index, i
          if i == steps -1:
            start = 0
          if i > 0 and i % 100 == 0:
            model.save(sess, save_path, global_step=i)
        model.save(sess, save_path, global_step=i)
      flow.close()

  # 测试部分
  tf.reset_default_graph()
  model_pred = SequenceToSequence(
    input_vocab_size= len(ws),
    target_vocab_size = len(ws),
    batch_size=1,
    mode='decode',
    beam_width=12,
    **params
  )
  init = tf.global_variables_initializer()

  with tf.Session(config=config) as sess:
    sess.run(init)
    model_pred.load(sess, ckpt)

    bar = batch_flow([x_data, y_data], ws, 1, add_end=False)
    t=0
    for x, xl, y, yl in bar:
      x = np.flip(x, axis=1)
      pred = model_pred.predict(
        sess,
        np.array(x),
        np.array(xl)
      )
      print(ws.inverse_transform(x[0]))
      print(ws.inverse_transform(y[0]))
      print(ws.inverse_transform(pred[0]))
      t+=1
      if t >= 3:
        break

def main():
  os.environ['CUDA_VISIBLE_DEVICES'] = '2'
  train(json.load(open('params.json')))
  
  
if __name__ == '__main__':
  main()          


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
try load model from ./model/s2s_chatbot_anti.ckpt-14700
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/s2s_chatb

epoch 1, loss=0.000000:  66%|██████▌   | 14700/22337 [00:00<?, ?it/s]

当前：14700


epoch 1 loss=2.336810 lr=0.000842:  66%|██████▌   | 14701/22337 [00:18<39:06:07, 18.43s/it]

当前：14701


...

当前：15949


epoch 1 loss=2.221609 lr=0.000827:  71%|███████▏  | 15950/22337 [1:44:45<9:20:43,  5.27s/it]

当前：15950


epoch 1 loss=2.221479 lr=0.000827:  71%|███████▏  | 15951/22337 [1:44:51<9:32:34,  5.38s/it]

当前：15951
